<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Gemini_2_5_Agentic_Flight_Planner_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import google.generativeai as genai
import time
import json
from google.colab import userdata # Keep this as per your instruction

# --- API Key Setup (as provided by you, directly used) ---
GOOGLE_API_KEY = userdata.get('GEMINI')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
else:
    print("WARNING: GOOGLE_API_KEY not found in Colab Secrets. Please ensure 'GEMINI' secret is set.")
    print("API calls will likely fail. Proceeding with unconfigured API.")

# --- Agent Configuration ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash" # As specified by you

# Initialize the Gemini model for general responses and agentic decisions
try:
    AGENTIC_MODEL = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
    RESPONDER_MODEL = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
    print(f"Gemini model '{AgentConfig.LLM_MODEL_NAME}' initialized for agentic and response generation.")
except Exception as e:
    print(f"ERROR: Failed to initialize Gemini model. Please check your API key and model name. Error: {e}")
    # Fallback to dummy models if Gemini initialization fails
    AGENTIC_MODEL = None
    RESPONDER_MODEL = None

# --- 1. Simulate a Knowledge Base ---
# This remains the same as it's an external data source, not the LLM.
KNOWLEDGE_BASE = {
    "flight_routes": "Common flight routes include New York to London (JFK-LHR), San Francisco to Tokyo (SFO-NRT), and Dubai to Sydney (DXB-SYD). These are typically optimized for shortest distance and fuel efficiency.",
    "aircraft_types": "Popular aircraft types are the Boeing 737 for short-haul, Boeing 747 and Airbus A380 for long-haul, and smaller regional jets like the Embraer E-Jets. Each has specific performance characteristics.",
    "weather_impact": "Adverse weather conditions such as heavy thunderstorms, strong crosswinds, or dense fog can cause flight delays or cancellations. Pilots receive detailed weather briefings.",
    "airport_codes": "Major airport codes include JFK (New York), LAX (Los Angeles), LHR (London Heathrow), CDG (Paris Charles de Gaulle), and DXB (Dubai International).",
    "flight_planning_basics": "Flight planning involves calculating fuel requirements, optimal altitude, route selection based on winds, and checking airspace restrictions. It's crucial for safe and efficient travel and involves detailed meteorological and aeronautical information.",
    "ai_in_aviation": "AI can optimize flight routes, predict maintenance needs, enhance air traffic control efficiency, and improve passenger experience in aviation by processing vast amounts of data quickly."
}

# --- 2. Simulate the Retriever ---
def simple_retriever(query: str, knowledge_base: dict, top_k: int = 1) -> list[str]:
    """
    Simulates retrieving relevant documents from the knowledge base based on keywords in the query.
    """
    print(f"\n  [Tool: Retriever] Searching knowledge base for '{query}'...")
    time.sleep(0.3) # Simulate retrieval time
    relevant_docs = []
    query_words = set(query.lower().split())

    for key, doc_text in knowledge_base.items():
        if any(word in key.lower() for word in query_words) or \
           any(word in doc_text.lower() for word in query_words):
            relevant_docs.append(doc_text)
            if len(relevant_docs) >= top_k:
                break

    if not relevant_docs:
        print("  [Tool: Retriever] No relevant documents found.")
    else:
        print(f"  [Tool: Retriever] Found {len(relevant_docs)} relevant document(s).")
    return relevant_docs

# --- 3. Gemini LLM for final response generation ---
def gemini_llm_responder(prompt: str) -> str:
    """
    Uses the actual Gemini model to generate a response.
    """
    if RESPONDER_MODEL is None:
        return "ERROR: Gemini model not initialized. Cannot respond."

    print(f"  [LLM Final Response] Processing prompt with {AgentConfig.LLM_MODEL_NAME}: '{prompt[:90]}...'")
    try:
        response = RESPONDER_MODEL.generate_content(prompt)
        # Assuming direct text response for simplicity. Handle parts if tool_code or function_call are expected.
        return response.text
    except Exception as e:
        print(f"ERROR: Gemini LLM response failed: {e}")
        return f"An error occurred while generating response: {e}"

# --- 4. Gemini LLM as Agentic Decision Maker ---
def gemini_agentic_decision_maker(current_state_prompt: str, iteration: int) -> tuple[str, str, str]:
    """
    Uses the actual Gemini model to decide the next action based on a structured prompt.
    Returns thought, action ('retrieve' or 'answer'), and retrieval_query (if action is 'retrieve').
    """
    if AGENTIC_MODEL is None:
        return "ERROR: Gemini model not initialized.", "error", ""

    print(f"\n  [Agentic LLM] Iteration {iteration}: Thinking with {AgentConfig.LLM_MODEL_NAME} about '{current_state_prompt[:70]}...'")

    # This prompt guides the LLM to output a structured JSON decision.
    system_instruction = (
        "You are an intelligent AI agent for flight planning. Your task is to analyze the user's "
        "query and the current conversation state, then decide on the best next action. "
        "You have access to a 'retriever' tool for external knowledge. "
        "Output your decision as a JSON object with two required keys: 'thought' (a string explaining your reasoning) "
        "and 'action' (a string, either 'retrieve' if you need more information, or 'answer' if you have enough information to provide a final response). "
        "If your 'action' is 'retrieve', you *must* also include a 'retrieval_query' (a string) which is the specific query you'd pass to the retriever tool. "
        "If your 'action' is 'answer', you should not include 'retrieval_query'."
    )
    user_query = f"User Query: {current_state_prompt}\n\nWhat is your next action?"

    try:
        response = AGENTIC_MODEL.generate_content(
            [system_instruction, user_query],
            generation_config=genai.types.GenerationConfig(response_mime_type="application/json") # Request JSON output
        )
        decision_json = json.loads(response.text) # Parse the JSON output

        thought = decision_json.get("thought", "No thought provided.")
        action = decision_json.get("action", "answer").lower()
        retrieval_query = decision_json.get("retrieval_query", "")

        print(f"  [Agentic LLM] Thought: {thought}")
        print(f"  [Agentic LLM] Decided Action: {action}")
        if action == "retrieve":
            print(f"  [Agentic LLM] Retrieval Query: '{retrieval_query}'")
        return thought, action, retrieval_query

    except Exception as e:
        print(f"ERROR: Gemini Agentic decision failed: {e}")
        print(f"LLM Raw Output (if available): {response.text if 'response' in locals() else 'N/A'}")
        return f"An error occurred while deciding: {e}", "error", ""

# --- 5. Agentic System Orchestration ---
def run_agentic_flight_planner(original_prompt: str, max_iterations: int = 2):
    """
    Demonstrates the agentic system flow for flight planning using Gemini.
    """
    print("\n" + "="*50)
    print("--- Running Agentic Flight Planner System (with Gemini 2.5) ---")
    print(f"  [User Prompt] Original: '{original_prompt}'")
    start_time = time.time()

    current_prompt_for_llm = original_prompt
    final_response = "Could not generate a response."
    retrieved_context = ""

    for i in range(max_iterations):
        thought, action, retrieval_query = gemini_agentic_decision_maker(current_prompt_for_llm, i)

        if action == "retrieve":
            relevant_docs = simple_retriever(retrieval_query if retrieval_query else original_prompt, KNOWLEDGE_BASE)
            if relevant_docs:
                retrieved_context = "\n\nContext from Knowledge Base:\n" + "\n".join([f"- {doc}" for doc in relevant_docs])
                current_prompt_for_llm = f"User Question: {original_prompt}\n{retrieved_context}\n\nBased on the above context, please provide a comprehensive answer to the original question."
                print(f"  [Agent] Prompt augmented for next LLM call with context.")
            else:
                print("  [Agent] No relevant documents found, proceeding without external context.")
                current_prompt_for_llm = f"User Question: {original_prompt}\n\nNo relevant external context found. Please answer based on your internal knowledge."

        elif action == "answer":
            final_response = gemini_llm_responder(current_prompt_for_llm)
            break

        else: # Covers "error" or other unexpected actions
            print(f"  [Agent] Unrecognized or error action: {action}. Exiting.")
            final_response = f"Agent encountered an issue: {thought}"
            break

    # Fallback if loop finishes without a direct answer action
    if final_response == "Could not generate a response.":
        print("  [Agent] Max iterations reached without a final answer decision. Attempting final response with gathered context.")
        final_response = gemini_llm_responder(current_prompt_for_llm)

    end_time = time.time()
    latency = (end_time - start_time) * 1000 # milliseconds
    print(f"\n  [Final Agent Response] {final_response}")
    print(f"--- Agentic Flight Planner System Finished (Latency: {latency:.2f} ms) ---")
    print("="*50 + "\n")
    return final_response, latency

# --- Normal LLM Use Orchestration (for comparison, now with Gemini) ---
def run_normal_llm_use(original_prompt: str):
    """
    Demonstrates the normal LLM use flow with Gemini.
    """
    print("\n--- Running Normal LLM Use (with Gemini 2.5, for comparison) ---")
    start_time = time.time()

    print(f"  [User Prompt] Original: '{original_prompt}'")
    time.sleep(0.1)

    response = gemini_llm_responder(original_prompt)

    end_time = time.time()
    latency = (end_time - start_time) * 1000 # milliseconds
    print(f"  [LLM Response] {response}")
    print(f"--- Normal LLM Use Finished (Latency: {latency:.2f} ms) ---")
    return response, latency

# --- Demo Execution ---
if __name__ == "__main__":
    print(f"Welcome to the Gemini 2.5 AI Agent Flight Planning Demo (EST).")
    print("This demo now uses a real Gemini 2.5 model for agentic decision-making and response generation.\n")
    print("Ensure your 'GEMINI' secret is set in Google Colab for the API key to be accessed.\n")

    # --- Demo 1: Agent decides to retrieve ---
    prompt1 = "What are the common flight routes and what factors optimize them?"
    agent_response1, agent_latency1 = run_agentic_flight_planner(prompt1)

    # --- Demo 1b: Single-Pass RAG with Gemini (for comparison) ---
    print("\n" + "="*50)
    print("--- Running Single-Pass RAG (with Gemini, for direct comparison to Agentic) ---")
    _start_time = time.time()
    _retrieval_query_for_rag = prompt1
    _relevant_docs_rag = simple_retriever(_retrieval_query_for_rag, KNOWLEDGE_BASE)
    _augmented_prompt_rag = prompt1
    if _relevant_docs_rag:
        _augmented_prompt_rag = f"{prompt1}\n\nContext from Knowledge Base:\n" + "\n".join([f"- {doc}" for doc in _relevant_docs_rag]) + "\n\nBased on the above context and my original question, please provide a comprehensive answer."
    _rag_response = gemini_llm_responder(_augmented_prompt_rag)
    _rag_latency = (time.time() - _start_time) * 1000
    print(f"  [Single-Pass RAG Response] {_rag_response}")
    print(f"--- Single-Pass RAG Finished (Latency: {_rag_latency:.2f} ms) ---")
    print("="*50 + "\n")


    # --- Demo 2: Agent might directly answer or retrieve based on LLM logic ---
    prompt2 = "Tell me about the basics of flight planning and how AI can improve safety and efficiency."
    agent_response2, agent_latency2 = run_agentic_flight_planner(prompt2)
    normal_response2, normal_latency2 = run_normal_llm_use(prompt2)

    print("\n" + "="*70 + "\n")

    # --- Demo 3: Agent decides to retrieve for specific query ---
    prompt3 = "What impact does adverse weather have on flights, and how do pilots handle it?"
    agent_response3, agent_latency3 = run_agentic_flight_planner(prompt3)
    normal_response3, normal_latency3 = run_normal_llm_use(prompt3)

    print("\n" + "="*70 + "\n")
    print("\n--- Summary of Latency Comparisons (milliseconds) ---")
    print(f"Prompt 1 (Routes): Agentic: {agent_latency1:.2f} ms | Single-Pass RAG: {_rag_latency:.2f} ms")
    print(f"Prompt 2 (Flight Planning + AI): Agentic: {agent_latency2:.2f} ms | Normal LLM: {normal_latency2:.2f} ms")
    print(f"Prompt 3 (Weather): Agentic: {agent_latency3:.2f} ms | Normal LLM: {normal_latency3:.2f} ms")

    print("\nThis revised version directly implements the API key setup you provided, ensuring accurate execution within your Colab environment. "
          "The Gemini 2.5 model now drives the agent's decisions to retrieve or respond, providing a dynamic and intelligent core for our flight planning AI agent in EST.")

Google Generative AI configured successfully using Colab Secrets.
Gemini model 'gemini-2.5-flash' initialized for agentic and response generation.
Welcome to the Gemini 2.5 AI Agent Flight Planning Demo (EST).
This demo now uses a real Gemini 2.5 model for agentic decision-making and response generation.

Ensure your 'GEMINI' secret is set in Google Colab for the API key to be accessed.


--- Running Agentic Flight Planner System (with Gemini 2.5) ---
  [User Prompt] Original: 'What are the common flight routes and what factors optimize them?'

  [Agentic LLM] Iteration 0: Thinking with gemini-2.5-flash about 'What are the common flight routes and what factors optimize them?...'
  [Agentic LLM] Thought: The user is asking for information about common flight routes and the factors that optimize them. This is a knowledge-based query that requires external information. I should use the 'retriever' tool to get this information.
  [Agentic LLM] Decided Action: retrieve
  [Agentic LLM] Retri